In [1]:
# Install TensorFlow
!pip install tensorflow

# Install Pandas
!pip install pandas

# Install Hugging Face Transformers
!pip install transformers

# Install NumPy
!pip install numpy


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import tensorflow as tf
import pandas as pd
from transformers import (
    DistilBertTokenizer, TFDistilBertForSequenceClassification,
    RobertaTokenizer, TFRobertaForSequenceClassification,
)
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

/storage/icds/RISE/sw8/anaconda/conda_envs/tensorflow/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
val_data = pd.read_csv("./test_data.csv")[["text", "label"]]
train_data = pd.read_csv("./train_data.csv")[["text", "label"]]
def clean(df):
    df['label'] = df['label'].map({'Real': True, 'Fake': False})
    return df
val_data = clean(val_data)
train_data = clean(train_data)

In [4]:
display(val_data)
display(train_data)

,text,label
0,Obama Blasts Trump At Veterans Event For Trash...,False
1,Thirteen Chinese fishermen die as boat collide...,True
2,Far-right presidential hopeful aims to be Braz...,True
3,"Defying warnings, residents refuse to leave Mu...",True
4,Anti-Uber protests disrupt major Chilean airpo...,True
...,...,...
4487,Boiler Room EP #130 Mandalay Cover-Up,False
4488,WATCH: HANOI JANE FONDA Had Chance To Expose S...,False
4489,Islamic State seizes new Afghan foothold after...,True
4490,Japan to impose additional sanctions on North ...,True


,text,label
0,Anthem to pare back Obamacare offerings in Nev...,True
1,IF THESE CELEBRITIES ARE With Her Then Why Is ...,False
2,VA chief presses Congress to make it easier to...,True
3,FALSE NARRATIVE EXPOSED: CASTILE WAS ARMED ROB...,False
4,Both pro and anti-Brexit lawmakers back oustin...,True
...,...,...
40418,Nigeria says U.S. agrees delayed $593 million ...,True
40419,VIDEO: Top 10 Most Embarrassing Presidential F...,False
40420,DELEGATES FOR DUMMIES: How Theyre AwardedAnd H...,False
40421,Republican tax plan would deal financial hit t...,True


In [4]:
# Load models and tokenizers as before
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('jojo0616/my_Misinformation_distilbert_model')
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('jojo0616/my_Misinformation_distilbert_model', from_pt=True)

roberta_tokenizer_v1 = RobertaTokenizer.from_pretrained('vikram71198/distilroberta-base-finetuned-fake-news-detection')
roberta_model_v1 = TFRobertaForSequenceClassification.from_pretrained('vikram71198/distilroberta-base-finetuned-fake-news-detection', from_pt=True)

roberta_tokenizer_v2 = RobertaTokenizer.from_pretrained('hamzab/roberta-fake-news-classification')
roberta_model_v2 = TFRobertaForSequenceClassification.from_pretrained('hamzab/roberta-fake-news-classification', from_pt=True)

2024-10-23 21:38:02.582681: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-23 21:38:07.191494: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38402 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:2b:00.0, compute capability: 8.6
2024-10-23 21:38:07.769109: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-10-23 21:38:10.804597: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
All PyTorch model weights were used when initializ

In [5]:
train_labels = train_data['label'].tolist()
train_text = train_data['text'].tolist()
val_labels = val_data['label'].tolist()
val_text = val_data['text'].tolist()

In [7]:
def train_and_save_model(model, tokenizer, texts, labels, model_save_path, epochs=3):
    # Tokenize texts
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="tf", max_length=512)
    
    # Convert labels to TensorFlow tensors
    labels = tf.convert_to_tensor(labels)

    # Create a TensorFlow dataset
    full_dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), labels)).shuffle(len(labels))

    # Calculate the number of samples for training (90%) and validation (10%)
    train_size = int(0.9 * len(labels))
    val_size = len(labels) - train_size

    # Split the dataset into training and validation
    train_dataset = full_dataset.take(train_size).batch(8)
    val_dataset = full_dataset.skip(train_size).batch(8)
    
    # Define initial learning rate
    initial_learning_rate = 5e-5
    
    # Define learning rate schedule
    def lr_scheduler(epoch, lr):
        if epoch < 10:
            return lr
        else:
            return lr * tf.math.exp(-0.1)
    
    # Learning Rate Scheduler callback
    scheduler = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)
    
    # Early Stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
    
    # Train the model with learning rate scheduler, early stopping, and the validation set
    model.fit(train_dataset, epochs=epochs, validation_data=val_dataset, callbacks=[scheduler, early_stopping])
    
    # Save the model and tokenizer
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

# Convert labels to TensorFlow tensors
labels_tensor = tf.convert_to_tensor(train_labels)

# No need to pre-tokenize the train_text. Pass raw text directly.
# Specify the save paths for each model
distilbert_save_path = './distilbert_finetuned_political'
roberta_v1_save_path = './roberta_v1_finetuned_political'
roberta_v2_save_path = './roberta_v2_finetuned_political'

# Train and save each model. Note that we're now passing `train_text` and `labels_tensor` directly.
train_and_save_model(distilbert_model, distilbert_tokenizer, train_text, labels_tensor, distilbert_save_path, epochs=5)
train_and_save_model(roberta_model_v1, roberta_tokenizer_v1, train_text, labels_tensor, roberta_v1_save_path, epochs=5)
train_and_save_model(roberta_model_v2, roberta_tokenizer_v2, train_text, labels_tensor, roberta_v2_save_path, epochs=5)

print("Models and tokenizers have been saved.")

Epoch 1/5
4548/4548 [==============================] - 184s 39ms/step - loss: 0.0830 - accuracy: 0.9699 - val_loss: 0.0242 - val_accuracy: 0.9936 - lr: 5.0000e-05
Epoch 2/5
4548/4548 [==============================] - 178s 39ms/step - loss: 0.0218 - accuracy: 0.9927 - val_loss: 0.0058 - val_accuracy: 0.9980 - lr: 5.0000e-05
Epoch 4/5
3165/4548 [===================>..........] - ETA: 51s - loss: 0.0150 - accuracy: 0.9952

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



4548/4548 [==============================] - 179s 39ms/step - loss: 0.0160 - accuracy: 0.9953 - val_loss: 0.0049 - val_accuracy: 0.9985 - lr: 5.0000e-05
Epoch 5/5
4548/4548 [==============================] - 363s 78ms/step - loss: 0.3565 - accuracy: 0.7587 - val_loss: 0.6921 - val_accuracy: 0.5293 - lr: 5.0000e-05
Epoch 2/5
4548/4548 [==============================] - 353s 78ms/step - loss: 0.6946 - accuracy: 0.5166 - val_loss: 0.6918 - val_accuracy: 0.5303 - lr: 5.0000e-05
Epoch 3/5
4548/4548 [==============================] - 352s 77ms/step - loss: 0.6841 - accuracy: 0.5274 - val_loss: 0.6960 - val_accuracy: 0.5271 - lr: 5.0000e-05
Epoch 4/5
4548/4548 [==============================] - 354s 78ms/step - loss: 0.4486 - accuracy: 0.6996 - val_loss: 0.1029 - val_accuracy: 0.9743 - lr: 5.0000e-05
Epoch 5/5
4548/4548 [==============================] - 354s 78ms/step - loss: 0.0953 - accuracy: 0.9771 - val_loss: 0.0791 - val_accuracy: 0.9819 - lr: 5.0000e-05
Models and tokenizers have been 

In [6]:
# Load the fine-tuned DistilBERT model and tokenizer
distilbert_tokenizer = DistilBertTokenizer.from_pretrained('./distilbert_finetuned_political')
distilbert_model = TFDistilBertForSequenceClassification.from_pretrained('./distilbert_finetuned_political')

# Load the fine-tuned RoBERTa model and tokenizer for the first variant
roberta_tokenizer_v1 = RobertaTokenizer.from_pretrained('./roberta_v1_finetuned_political')
roberta_model_v1 = TFRobertaForSequenceClassification.from_pretrained('./roberta_v1_finetuned_political')

# Load the fine-tuned RoBERTa model and tokenizer for the second variant
roberta_tokenizer_v2 = RobertaTokenizer.from_pretrained('./roberta_v2_finetuned_political')
roberta_model_v2 = TFRobertaForSequenceClassification.from_pretrained('./roberta_v2_finetuned_political')

def ensemble_classify_news_and_evaluate_accuracy(df):
    # Lists to store individual model predictions
    distilbert_predictions = []
    roberta_v1_predictions = []
    roberta_v2_predictions = []
    ensemble_predictions = []
    
    for _, row in df.iterrows():
        text_input = row['text']

        # Prepare inputs and get probabilities for DistilBERT
        distilbert_inputs = distilbert_tokenizer(text_input, return_tensors="tf", truncation=True, padding='max_length', max_length=512)
        distilbert_outputs = distilbert_model(distilbert_inputs)
        distilbert_probabilities = tf.nn.softmax(distilbert_outputs.logits, axis=-1)
        distilbert_predicted_class_index = tf.argmax(distilbert_probabilities, axis=-1).numpy()[0]
        distilbert_predictions.append(True if distilbert_predicted_class_index == 1 else False)

        # Prepare inputs and get probabilities for RoBERTa variant 1
        roberta_inputs_v1 = roberta_tokenizer_v1(text_input, return_tensors="tf", truncation=True, padding='max_length', max_length=512)
        roberta_outputs_v1 = roberta_model_v1(roberta_inputs_v1)
        roberta_probabilities_v1 = tf.nn.softmax(roberta_outputs_v1.logits, axis=-1)
        roberta_v1_predicted_class_index = tf.argmax(roberta_probabilities_v1, axis=-1).numpy()[0]
        roberta_v1_predictions.append(True if roberta_v1_predicted_class_index == 1 else False)

        # Prepare inputs and get probabilities for RoBERTa variant 2
        roberta_inputs_v2 = roberta_tokenizer_v2(text_input, return_tensors="tf", truncation=True, padding='max_length', max_length=512)
        roberta_outputs_v2 = roberta_model_v2(roberta_inputs_v2)
        roberta_probabilities_v2 = tf.nn.softmax(roberta_outputs_v2.logits, axis=-1)
        roberta_v2_predicted_class_index = tf.argmax(roberta_probabilities_v2, axis=-1).numpy()[0]
        roberta_v2_predictions.append(True if roberta_v2_predicted_class_index == 1 else False)

        # Ensemble: Average the probabilities from all models
        avg_probabilities = (distilbert_probabilities + roberta_probabilities_v1 + roberta_probabilities_v2) / 3
        predicted_class_index = tf.argmax(avg_probabilities, axis=-1).numpy()[0]
        ensemble_predictions.append(True if predicted_class_index == 1 else False)

    # Adding predictions to the DataFrame
    df['DistilBERTPrediction'] = distilbert_predictions
    df['RoBERTaV1Prediction'] = roberta_v1_predictions
    df['RoBERTaV2Prediction'] = roberta_v2_predictions
    df['EnsemblePrediction'] = ensemble_predictions
    
    # Calculate and print the accuracy for the ensemble predictions
    correct_predictions = (df['EnsemblePrediction'] == df['label']).sum()
    total_predictions = len(df)
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy: {accuracy:.4f}")
    
    return df

Some layers from the model checkpoint at ./distilbert_finetuned_political were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at ./distilbert_finetuned_political and are newly initialized: ['dropout_97']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
All model checkpoint layers were used when

In [13]:
df_updated = ensemble_classify_news_and_evaluate_accuracy(val_data)
display(df_updated)

Accuracy: 0.9931


,text,label,DistilBERTPrediction,RoBERTaV1Prediction,RoBERTaV2Prediction,EnsemblePrediction
0,Obama Blasts Trump At Veterans Event For Trash...,False,False,False,False,False
1,Thirteen Chinese fishermen die as boat collide...,True,True,True,True,True
2,Far-right presidential hopeful aims to be Braz...,True,True,True,True,True
3,"Defying warnings, residents refuse to leave Mu...",True,True,True,True,True
4,Anti-Uber protests disrupt major Chilean airpo...,True,True,True,True,True
...,...,...,...,...,...,...
4487,Boiler Room EP #130 Mandalay Cover-Up,False,False,False,True,False
4488,WATCH: HANOI JANE FONDA Had Chance To Expose S...,False,False,False,False,False
4489,Islamic State seizes new Afghan foothold after...,True,True,True,True,True
4490,Japan to impose additional sanctions on North ...,True,True,True,True,True


In [10]:
prediction_columns = ['DistilBERTPrediction', 'RoBERTaV1Prediction', 'RoBERTaV2Prediction', 'EnsemblePrediction']

# Loop through each prediction column to calculate and print accuracy
for column in prediction_columns:
    correct_predictions = (df_updated[column] == df_updated['label']).sum()
    total_predictions = len(df_updated)
    accuracy = correct_predictions / total_predictions
    print(f"Accuracy for {column}: {accuracy:.4f}")

Accuracy for DistilBERTPrediction: 0.9850
Accuracy for RoBERTaV1Prediction: 1.0000
Accuracy for RoBERTaV2Prediction: 0.9800
Accuracy for EnsemblePrediction: 0.9950


In [11]:
df_updated.rename(columns={
    'DistilBERTPrediction': 'DistilBERTPrediction_Text',
    'RoBERTaV1Prediction': 'RoBERTaV1Prediction_Text',
    'RoBERTaV2Prediction': 'RoBERTaV2Prediction_Text',
    'EnsemblePrediction': 'EnsemblePrediction_Text'
}, inplace=True)
df_updated.to_csv("./Constraint_Val_Labeled.csv")

/tmp/ipykernel_759123/1222044664.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_updated.rename(columns={
